In [8]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [10]:
#Cargar los datos de entrenamiento y test
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [12]:
train.columns

Index(['TP2_mean', 'TP2_max', 'TP2_min', 'TP2_median', 'TP2_var', 'TP3_mean',
       'TP3_max', 'TP3_min', 'TP3_median', 'TP3_var', 'H1_mean', 'H1_max',
       'H1_min', 'H1_median', 'H1_var', 'DV_pressure_mean', 'DV_pressure_max',
       'DV_pressure_min', 'DV_pressure_median', 'DV_pressure_var',
       'Reservoirs_mean', 'Reservoirs_max', 'Reservoirs_min',
       'Reservoirs_median', 'Reservoirs_var', 'Oil_temperature_mean',
       'Oil_temperature_max', 'Oil_temperature_min', 'Oil_temperature_median',
       'Oil_temperature_var', 'Motor_current_mean', 'Motor_current_max',
       'Motor_current_min', 'Motor_current_median', 'Motor_current_var',
       'COMP_mean', 'COMP_max', 'COMP_min', 'COMP_median', 'COMP_var',
       'DV_eletric_mean', 'DV_eletric_max', 'DV_eletric_min',
       'DV_eletric_median', 'DV_eletric_var', 'Towers_mean', 'Towers_max',
       'Towers_min', 'Towers_median', 'Towers_var', 'MPG_mean', 'MPG_max',
       'MPG_min', 'MPG_median', 'MPG_var', 'LPS_mean', 'LPS_m

In [14]:
test.columns

Index(['TP2_mean', 'TP2_max', 'TP2_min', 'TP2_median', 'TP2_var', 'TP3_mean',
       'TP3_max', 'TP3_min', 'TP3_median', 'TP3_var', 'H1_mean', 'H1_max',
       'H1_min', 'H1_median', 'H1_var', 'DV_pressure_mean', 'DV_pressure_max',
       'DV_pressure_min', 'DV_pressure_median', 'DV_pressure_var',
       'Reservoirs_mean', 'Reservoirs_max', 'Reservoirs_min',
       'Reservoirs_median', 'Reservoirs_var', 'Oil_temperature_mean',
       'Oil_temperature_max', 'Oil_temperature_min', 'Oil_temperature_median',
       'Oil_temperature_var', 'Motor_current_mean', 'Motor_current_max',
       'Motor_current_min', 'Motor_current_median', 'Motor_current_var',
       'COMP_mean', 'COMP_max', 'COMP_min', 'COMP_median', 'COMP_var',
       'DV_eletric_mean', 'DV_eletric_max', 'DV_eletric_min',
       'DV_eletric_median', 'DV_eletric_var', 'Towers_mean', 'Towers_max',
       'Towers_min', 'Towers_median', 'Towers_var', 'MPG_mean', 'MPG_max',
       'MPG_min', 'MPG_median', 'MPG_var', 'LPS_mean', 'LPS_m

In [16]:
# Definir los parámetros a buscar en el grid para el modelo SVM
param_grid = {
    'C': [0.1, 1, 10],       # Parámetro de penalización
    'gamma': ['scale', 'auto', 0.1], 
    'kernel': ['rbf', 'linear']  # Tipos de kernel
}

# Crear un objeto StratifiedKFold para usar los folds
skf = StratifiedKFold(n_splits=train['fold'].nunique(), shuffle=True, random_state=42)

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid,
    scoring='accuracy',
    cv=skf,  # Folds para la validación cruzada
    n_jobs=-1, 
    verbose=1
)

# Entrenar el modelo con la búsqueda de hiperparámetros
X_train = train.drop(columns=['is_anomaly', 'fold'])
y_train = train['is_anomaly']

grid_search.fit(X_train, y_train)

# Evaluar el mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
X_test = test.drop(columns=['is_anomaly'])
y_test = test['is_anomaly']

y_pred_test = best_model.predict(X_test)

print("Best Parameters:", grid_search.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred_test))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_test))

Fitting 4 folds for each of 18 candidates, totalling 72 fits
Best Parameters: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}

Classification Report:
               precision    recall  f1-score   support

       False       0.99      1.00      0.99       557
        True       1.00      0.77      0.87        30

    accuracy                           0.99       587
   macro avg       0.99      0.88      0.93       587
weighted avg       0.99      0.99      0.99       587


Confusion Matrix:
 [[557   0]
 [  7  23]]
